In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie

  # CairoMakie's display() in PDF format opens an interactive window
  # instead of saving to the ipynb file, so we don't do that.
  # https://github.com/quarto-dev/quarto-cli/issues/7548
  if fig_format == :pdf
    CairoMakie.activate!(type = "png")
  else
    CairoMakie.activate!(type = string(fig_format))
  end
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/home/vituri/Documentos/GitHub/exploring_julia"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
[1, 2, 3]

3-element Vector{Int64}:
 1
 2
 3

In [3]:
[1:7;]

7-element Vector{Int64}:
 1
 2
 3
 4
 5
 6
 7

In [4]:
[1:2:11;]

6-element Vector{Int64}:
  1
  3
  5
  7
  9
 11

In [5]:
v = [1:2:999;]
sum(v)

250000

In [6]:
v = ["julia", "is", "nice!"]

3-element Vector{String}:
 "julia"
 "is"
 "nice!"

In [7]:
sort(v)

3-element Vector{String}:
 "is"
 "julia"
 "nice!"

In [8]:
[x^2 for x ∈ 1:5]

5-element Vector{Int64}:
  1
  4
  9
 16
 25

In [9]:
[x^2 for x ∈ 1:10 if (x % 2 == 0)]

5-element Vector{Int64}:
   4
  16
  36
  64
 100

In [10]:
v = [x for x ∈ 1:999 if (x % 3 == 0 || x % 5 == 0)]
sum(v)

233168

In [11]:
square(x::Integer) = x^2;

In [12]:
v = [1:5;]

5-element Vector{Int64}:
 1
 2
 3
 4
 5

In [13]:
[square(x) for x ∈ v]

5-element Vector{Int64}:
  1
  4
  9
 16
 25

In [14]:
map(square, v)

5-element Vector{Int64}:
  1
  4
  9
 16
 25

In [15]:
square.(v)

5-element Vector{Int64}:
  1
  4
  9
 16
 25

In [16]:
[1:5;] .* 5

5-element Vector{Int64}:
  5
 10
 15
 20
 25

In [17]:
sum(1:100)^2 - sum((1:100).^2)

25164150

In [18]:
"""
Return a vector with all primes up to `n` using the
sieve of Eratosthenes algorithm.
"""
function sieve_of_eratosthenes(n::Integer)
    sieve = trues(n)  # Create a boolean vector to mark primes
    
    # Set the multiples of primes as false (not prime)
    for i in 2:isqrt(n) # iterate until the integer part of sqrt(2)

    # if the number is not a multiple,
      if sieve[i] == true

      # iterate trough its multiples and set then to false
        for j in i^2:i:n
          sieve[j] = false
        end
      end
    end
    
    primes = filter(x -> sieve[x], 2:n)  # Filter out non-primes
    
    return primes
end;

In [19]:
sieve_of_eratosthenes(100) |> string

"[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]"

In [20]:
function next_step(n::Integer)
    n == 1 && return(1)

    if iseven(n)        
        div(n, 2)
    else
        3*n + 1
    end
end

next_step (generic function with 1 method)

In [21]:
next_step(3)

10

In [22]:
Dict("a" => 1)

Dict{String, Int64} with 1 entry:
  "a" => 1